In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

In [2]:
train_data0 = np.load('data0.npy')
train_lab0 = np.load('lab0.npy')

train_data1 = np.load('data1.npy')
train_lab1 = np.load('lab1.npy')

train_data2 = np.load('data2.npy')
train_lab2 = np.load('lab2.npy')

final_train = np.concatenate((train_data0 , train_data1 , train_data2) , axis=0)
final_labels = np.concatenate((train_lab0,train_lab1,train_lab2), axis=0)




X_train, X_val, y_train, y_val = train_test_split(final_train, final_labels, test_size=0.2, random_state=42)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [4]:
class MyDataset(Dataset):
    def __init__(self, images, labels):
        self.images = torch.FloatTensor(images.reshape(-1, 1, 40, 168)) / 255.0
        self.labels = torch.FloatTensor(labels)

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [5]:
train_dataset = MyDataset(X_train, y_train)
val_dataset = MyDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [11]:
class AdvancedDigitSumCNN(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(AdvancedDigitSumCNN, self).__init__()
        
       
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        
        self.attention = nn.Sequential(
            nn.Conv2d(128, 1, kernel_size=1),
            nn.Sigmoid()
        )
        
        
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        
        
        self.fc1 = nn.Linear(128, 256)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(256, 128)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(128, 1)
        
    def forward(self, x):
        
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn3(self.conv3(x)))
        
       
        attention_weights = self.attention(x)
        x = x * attention_weights
        
        
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        
        
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        
        return x



## Added focal l1 loss cause read that it might be better for such a task.
class FocalL1Loss(nn.Module):
    def __init__(self, alpha=2.0, beta=0.25, epsilon=1e-6):
        super().__init__()
        self.alpha = alpha
        self.beta = beta
        self.epsilon = epsilon
        
    def forward(self, pred, target):
        l1_loss = torch.abs(pred - target)
        l1_loss = torch.clamp(l1_loss, min=self.epsilon)  
        focal_factor = torch.pow(l1_loss, self.alpha)
        loss = focal_factor * l1_loss
        return loss.mean()


In [14]:
def train_model_adv(model, train_loader, val_loader, criterion, optimizer, device, epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    best_val_loss = float('inf')
    best_model_path = "advmodel.pth"

    for epoch in range(epochs):
        
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)

        train_loss /= len(train_loader.dataset)

        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs.squeeze(), labels)
                val_loss += loss.item() * images.size(0)
        val_loss /= len(val_loader.dataset)

        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"Best model saved with Val Loss: {val_loss:.4f}")

    print("Training complete. Best model saved as advmodel.pth.")




In [15]:
advmodel = AdvancedDigitSumCNN()
advcriterion = FocalL1Loss()
optimizer = optim.Adam(advmodel.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
train_model_adv(advmodel, train_loader, val_loader, advcriterion, optimizer, device , epochs=50)



Epoch 1/50, Train Loss: 177.1652, Val Loss: 146.0568
Best model saved with Val Loss: 146.0568
Epoch 2/50, Train Loss: 123.5872, Val Loss: 46.3448
Best model saved with Val Loss: 46.3448
Epoch 3/50, Train Loss: 100.7178, Val Loss: 66.7292
Epoch 4/50, Train Loss: 90.2490, Val Loss: 22.0596
Best model saved with Val Loss: 22.0596
Epoch 5/50, Train Loss: 85.5029, Val Loss: 23.4490
Epoch 6/50, Train Loss: 76.7190, Val Loss: 30.3449
Epoch 7/50, Train Loss: 70.8224, Val Loss: 51.3272
Epoch 8/50, Train Loss: 63.1695, Val Loss: 16.7730
Best model saved with Val Loss: 16.7730
Epoch 9/50, Train Loss: 59.7111, Val Loss: 46.0979
Epoch 10/50, Train Loss: 55.2708, Val Loss: 31.6795
Epoch 11/50, Train Loss: 49.8542, Val Loss: 20.4186
Epoch 12/50, Train Loss: 45.6918, Val Loss: 12.1415
Best model saved with Val Loss: 12.1415
Epoch 13/50, Train Loss: 43.2284, Val Loss: 12.1626
Epoch 14/50, Train Loss: 41.1865, Val Loss: 10.0756
Best model saved with Val Loss: 10.0756
Epoch 15/50, Train Loss: 38.2774, Va

In [18]:
advmodel.eval()
example_images, actual_labels = next(iter(val_loader))  
example_images = example_images.to(device) 


predictions = advmodel(example_images).detach().cpu().numpy()
rounded_pred = predictions.round()


for i in range(10): 
    print(f"Prediction: {rounded_pred[i][0]:.2f}, Actual: {actual_labels[i].item():.2f}")


Prediction: 15.00, Actual: 15.00
Prediction: 17.00, Actual: 18.00
Prediction: 18.00, Actual: 19.00
Prediction: 26.00, Actual: 26.00
Prediction: 24.00, Actual: 25.00
Prediction: 13.00, Actual: 14.00
Prediction: 10.00, Actual: 10.00
Prediction: 24.00, Actual: 23.00
Prediction: 17.00, Actual: 19.00
Prediction: 18.00, Actual: 20.00


In [33]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResNetSumPredictor(nn.Module):
    def __init__(self, pretrained=True):
        super(ResNetSumPredictor, self).__init__()

        self.resnet = models.resnet50(pretrained=pretrained)


        self.resnet.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=64,
            kernel_size=7,
            stride=2,
            padding=3,
            bias=False
        )

        if pretrained:
            state_dict = models.resnet50(pretrained=True).state_dict()

            state_dict['conv1.weight'] = state_dict['conv1.weight'].sum(dim=1, keepdim=True)
            self.resnet.load_state_dict(state_dict, strict=False)



        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, 1)  

    def forward(self, x):
        return self.resnet(x)


resmodel = ResNetSumPredictor(pretrained=True)



criterion = nn.MSELoss()
optimizer = torch.optim.Adam(resmodel.parameters(), lr=0.001)






In [34]:
def train_model_res(model, train_loader, val_loader, criterion, optimizer, device , epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    best_val_loss = float('inf')
    best_model_path = "resmodel.pth"
    for epoch in range(epochs):
        
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)

        train_loss /= len(train_loader.dataset)

        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs.squeeze(), labels)
                val_loss += loss.item() * images.size(0)
        val_loss /= len(val_loader.dataset)

        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"Best model saved with Val Loss: {val_loss:.4f}")

    print("Training complete. Best model saved as resmodel.pth.")

In [35]:
train_model_res(resmodel , train_loader , val_loader, criterion,optimizer,device,epochs=50)

Epoch 1/50, Train Loss: 7.1914, Val Loss: 9.4816
Best model saved with Val Loss: 9.4816
Epoch 2/50, Train Loss: 2.5557, Val Loss: 3.0220
Best model saved with Val Loss: 3.0220
Epoch 3/50, Train Loss: 1.9616, Val Loss: 1.1972
Best model saved with Val Loss: 1.1972
Epoch 4/50, Train Loss: 1.4332, Val Loss: 2.0275
Epoch 5/50, Train Loss: 1.1669, Val Loss: 1.4186
Epoch 6/50, Train Loss: 1.1432, Val Loss: 1.3201
Epoch 7/50, Train Loss: 0.8383, Val Loss: 0.7575
Best model saved with Val Loss: 0.7575
Epoch 8/50, Train Loss: 0.8641, Val Loss: 0.8375
Epoch 9/50, Train Loss: 0.6973, Val Loss: 0.8223
Epoch 10/50, Train Loss: 0.6886, Val Loss: 0.7765
Epoch 11/50, Train Loss: 0.5284, Val Loss: 0.8024
Epoch 12/50, Train Loss: 0.5540, Val Loss: 0.6732
Best model saved with Val Loss: 0.6732
Epoch 13/50, Train Loss: 0.5098, Val Loss: 0.8748
Epoch 14/50, Train Loss: 0.4418, Val Loss: 0.6863
Epoch 15/50, Train Loss: 0.5415, Val Loss: 0.5574
Best model saved with Val Loss: 0.5574
Epoch 16/50, Train Loss: 